In [29]:
import scipy
import numpy as np
import pandas as pd
import plotly.plotly as py
# import visplots
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from sklearn import preprocessing, metrics
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats.distributions import randint
import plotly.graph_objs as go
import plotly.tools as tls
tls.set_credentials_file(username='cap7carps', api_key='d28TqBXnTSA80L2xctmI')
from numpy import arange,array,ones
from scipy import stats
import datetime as dt

In [4]:
weatherFrame = pd.read_pickle(r'C:\GIT\weatherproj\Weather\weather_data.pki')

In [5]:
lonrain = pd.read_pickle( r'C:\GIT\weatherproj\Weather\lonrainpickle.pki')

In [6]:
glasrain = pd.read_pickle(r'C:\GIT\weatherproj\Weather\glasrainpickle')

In [7]:
weatherFrame.head()

,avg_rain_prob,avg_temp_frc,city,date_time_period,forecast_period,avg_temp_obs,avg_ws_obs,w_observed,avg_temp_dif,rain_accuracy
0,9.0,7.0,glasgow,2017-01-03 06:00:00,-1.0,7.3,9.5,"[8.0, 8.0]",0.3,0.91
1,9.0,6.0,glasgow,2017-01-03 06:00:00,2.0,7.3,9.5,"[8.0, 8.0]",1.3,0.91
2,8.0,6.0,glasgow,2017-01-03 06:00:00,5.0,7.3,9.5,"[8.0, 8.0]",1.3,0.92
3,10.0,6.0,glasgow,2017-01-03 06:00:00,8.0,7.3,9.5,"[8.0, 8.0]",1.3,0.90
4,9.0,6.0,glasgow,2017-01-03 06:00:00,11.0,7.3,9.5,"[8.0, 8.0]",1.3,0.91


In [42]:
weatherFrame2 = weatherFrame.ix[weatherFrame['forecast_period']==12.0]
weatherFrame2.head()
weatherFrame3 = weatherFrame2.sort_values(by='date_time_period')
weatherFlon = weatherFrame3.ix[weatherFrame['city']=='london']
weatherFglas = weatherFrame3.ix[weatherFrame['city']=='glasgow']

In [43]:
weatherFlon2 = weatherFlon.ix[:,['date_time_period', 'forecast_period', 'rain_accuracy', 'city']]
weatherFglas2 = weatherFglas.ix[:,['date_time_period', 'forecast_period', 'rain_accuracy', 'city']]

In [70]:
weatherFlon2.tail()

,date_time_period,forecast_period,rain_accuracy,city
857,2017-01-02 03:00:00,12.0,0.9880,london
797,2017-01-02 06:00:00,12.0,1.0000,london
496,2017-01-02 15:00:00,12.0,0.9775,london
396,2017-01-02 18:00:00,12.0,1.0000,london
72,2017-01-03 06:00:00,12.0,0.9480,london


In [45]:
weatherFglas2.head()

,date_time_period,forecast_period,rain_accuracy,city
17593,2016-12-09 09:00:00,12.0,0.44,glasgow
17572,2016-12-09 12:00:00,12.0,0.82,glasgow
17289,2016-12-09 18:00:00,12.0,0.14,glasgow
17180,2016-12-10 00:00:00,12.0,0.91,glasgow
16975,2016-12-10 06:00:00,12.0,1.00,glasgow


In [51]:
glas2rain = glasrain.ix[:,['date_time_period','text']].groupby('date_time_period').agg('count')

In [47]:
glasrain.head()

,Cold,Rain,W1,W2,Warm,Windy,_id,city,coordinates,created_at,date_time_period,text
0,N,Y,Y,Y,N,Y,586ce27c13dc7211f4de70ca,Glasgow,"{u'type': u'Point', u'coordinates': [-3.143, 5...",2016-12-08 15:00:47,2016-12-08 15:00:00,"15:00 GMT: Temperature: -, Wind: , Humidity: ,..."
1,N,Y,Y,Y,N,Y,586ce27c13dc7211f4de72fb,Glasgow,"{u'type': u'Point', u'coordinates': [-3.143, 5...",2016-12-08 16:03:11,2016-12-08 15:00:00,"16:00 GMT: Temperature: -, Wind: , Humidity: ,..."
2,N,Y,Y,Y,N,Y,586ce27c13dc7211f4de7568,Glasgow,"{u'type': u'Point', u'coordinates': [-3.143, 5...",2016-12-08 17:01:37,2016-12-08 15:00:00,"17:00 GMT: Temperature: -, Wind: , Humidity: ,..."
3,N,Y,Y,Y,N,Y,586ce27d13dc7211f4de7838,Glasgow,"{u'type': u'Point', u'coordinates': [-3.143, 5...",2016-12-08 18:01:37,2016-12-08 18:00:00,"18:00 GMT: Temperature: -, Wind: , Humidity: ,..."
4,N,Y,Y,Y,N,Y,586ce27d13dc7211f4de7ace,Glasgow,"{u'type': u'Point', u'coordinates': [-3.143, 5...",2016-12-08 19:01:37,2016-12-08 18:00:00,"19:00 GMT: Temperature: -, Wind: , Humidity: ,..."


In [9]:
lonrain.head()

,Cold,Rain,W1,W2,Warm,Windy,_id,city,coordinates,created_at,date_time_period,text
0,Y,Y,Y,Y,N,N,586b64e08d709f1f100a0500,london,"{u'type': u'Point', u'coordinates': [-0.145654...",2016-12-07 18:42:38,2016-12-07 18:00:00,"I'm at The Ice Wharf (Wetherspoon) in Camden, ..."
1,N,Y,Y,Y,N,N,586b64e18d709f1f100a096e,london,None,2016-12-07 18:59:18,2016-12-07 18:00:00,#thfc come rain or shine #derbyshireyids https...
2,N,Y,Y,Y,N,Y,586b64e18d709f1f100a0979,london,"{u'type': u'Point', u'coordinates': [-0.237222...",2016-12-07 18:59:27,2016-12-07 18:00:00,"Changeable, some precipitation. Now:12.1C - Ra..."
3,N,Y,Y,Y,N,Y,586b64e18d709f1f100a09d2,london,"{u'type': u'Point', u'coordinates': [-0.469791...",2016-12-07 19:00:42,2016-12-07 18:00:00,"18:52 Temp. 11.1°C, Hum. 85%, Dewp. 8°C, Bar. ..."
4,Y,Y,Y,Y,N,N,586b64e28d709f1f100a1202,london,None,2016-12-07 19:29:24,2016-12-07 18:00:00,It is snowing #hohoho https://t.co/r11qGWf0LK


In [52]:
lon2rain = lonrain.ix[:,['date_time_period','text']].groupby('date_time_period').agg('count')

In [69]:
lon2rain.tail()

,text
date_time_period,
2017-01-02 18:00:00,12
2017-01-02 21:00:00,11
2017-01-03 00:00:00,8
2017-01-03 03:00:00,8
2017-01-03 06:00:00,5


In [58]:
weatherFglas2.head()

,date_time_period,forecast_period,rain_accuracy,city
17593,2016-12-09 09:00:00,12.0,0.44,glasgow
17572,2016-12-09 12:00:00,12.0,0.82,glasgow
17289,2016-12-09 18:00:00,12.0,0.14,glasgow
17180,2016-12-10 00:00:00,12.0,0.91,glasgow
16975,2016-12-10 06:00:00,12.0,1.00,glasgow


In [59]:
glas2rain.head()

,text
date_time_period,
2016-12-08 15:00:00,3
2016-12-08 18:00:00,3
2016-12-08 21:00:00,5
2016-12-09 00:00:00,4
2016-12-09 03:00:00,3


In [60]:
glasfinal = pd.merge(weatherFglas2, glas2rain, left_on='date_time_period', right_index=True)

In [62]:
glasfinal.head()

,date_time_period,forecast_period,rain_accuracy,city,text
17593,2016-12-09 09:00:00,12.0,0.44,glasgow,7
17572,2016-12-09 12:00:00,12.0,0.82,glasgow,4
17289,2016-12-09 18:00:00,12.0,0.14,glasgow,4
17180,2016-12-10 00:00:00,12.0,0.91,glasgow,3
16660,2016-12-11 18:00:00,12.0,0.94,glasgow,2


In [63]:
lonfinal = pd.merge(weatherFlon2, lon2rain, left_on='date_time_period', right_index=True)

In [72]:
lonfinal

,date_time_period,forecast_period,rain_accuracy,city,text
18120,2016-12-08 15:00:00,12.0,0.900000,london,20
18067,2016-12-08 18:00:00,12.0,0.890000,london,21
18022,2016-12-08 21:00:00,12.0,0.910000,london,13
17872,2016-12-09 00:00:00,12.0,0.100000,london,8
17792,2016-12-09 03:00:00,12.0,0.220000,london,6
17750,2016-12-09 06:00:00,12.0,0.870000,london,11
17694,2016-12-09 09:00:00,12.0,0.960000,london,14
17497,2016-12-09 12:00:00,12.0,0.960000,london,13
17299,2016-12-09 18:00:00,12.0,0.952500,london,17
17432,2016-12-09 21:00:00,12.0,0.915000,london,15


In [67]:
list(glasfinal.date_time_period)

[Timestamp('2016-12-09 09:00:00'),
 Timestamp('2016-12-09 12:00:00'),
 Timestamp('2016-12-09 18:00:00'),
 Timestamp('2016-12-10 00:00:00'),
 Timestamp('2016-12-11 18:00:00'),
 Timestamp('2016-12-12 18:00:00'),
 Timestamp('2016-12-13 09:00:00'),
 Timestamp('2016-12-13 18:00:00'),
 Timestamp('2016-12-16 03:00:00'),
 Timestamp('2016-12-16 06:00:00'),
 Timestamp('2016-12-16 15:00:00'),
 Timestamp('2016-12-17 15:00:00'),
 Timestamp('2016-12-17 18:00:00'),
 Timestamp('2016-12-18 15:00:00'),
 Timestamp('2016-12-19 06:00:00'),
 Timestamp('2016-12-19 15:00:00')]

In [73]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Scatter(
    x = list(lonfinal.date_time_period),
    y = list(lonfinal.rain_accuracy.values),
    name='Wind Speed'
)
trace2 = go.Scatter(
    x = list(lonfinal.date_time_period),
    y = list(lonfinal.text.values),
    name='Count of Weather Related Tweets',
    yaxis='y2'
)
data = [trace1, trace2]
layout = go.Layout(
    title='Rain Forecast Accuracy vs Rainy Tweets',
    yaxis=dict(
        title='Accuracy (%)'
    ),
    yaxis2=dict(
        title='Count of Tweets',
        titlefont=dict(
            color='rgb(0, 100, 200)'
        ),        
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)